In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
yesterday = datetime.now() - timedelta(1)
input_csv_filename = yesterday.strftime("%d-%m-%Y") + "_jobs.csv"
df = pd.read_csv(input_csv_filename)


In [3]:
df['Date Posted'] = pd.to_datetime(df['Date Posted'], format='%d/%m/%Y')


In [4]:
replacements = {
    'T2': 'Thứ 2',
    'T3': 'Thứ 3',
    'T4': 'Thứ 4',
    'T5': 'Thứ 5',
    'T6': 'Thứ 6',
    'T7': 'Thứ 7',
    'CN': 'Chủ Nhật',
    ',':  'và',
    '-': 'đến'
}

# Thay thế trong cột 'Work Date'
df['Work Date'] = df['Work Date'].replace(replacements, regex=True)

In [5]:
df['Experience'] = df['Experience'].replace('Không yêu cầu', '0')
df['Experience'] = pd.to_numeric(df['Experience'], errors='coerce')

In [6]:
df[['Industry', 'Industry1']] = df['Industry'].str.split(' > ', expand=True)

In [ ]:
def extract_age_range(age_str):
    if age_str == "Không hiển thị":
        return 0, 100
    else:
        try:
            age_min, age_max = age_str.split("-")
            return int(age_min), int(age_max)
        except:
            return 0, 100  # In case the format is not expected

# Apply the function to the "Age" column and create new "Age Min" and "Age Max" columns
df[['Age Min', 'Age Max']] = df['Age'].apply(lambda x: pd.Series(extract_age_range(x)))

# Drop the original 'Age' column
df = df.drop(columns=['Age'])

In [9]:
def standardize_time(time_str):
    if isinstance(time_str, str):
        # Thay thế tất cả các kiểu A.M., P.M., a.m., p.m. thành AM, PM
        time_str = time_str.replace('A.M.', 'AM').replace('P.M.', 'PM').replace('a.m.', 'AM').replace('p.m.', 'PM')
        
        # Thay thế các kiểu chữ hoa "A.M" và "P.M" viết hoa
        time_str = time_str.replace(' A.M', ' AM').replace(' P.M', ' PM')

        # Xử lý trường hợp AM/PM
        if 'AM' in time_str or 'PM' in time_str:
            try:
                start, end = time_str.split(' - ')
                start = datetime.strptime(start, '%I:%M %p').strftime('%H:%M')
                end = datetime.strptime(end, '%I:%M %p').strftime('%H:%M')
                return f'{start} - {end}'
            except ValueError:
                return np.nan
        # Xử lý định dạng giờ 24h (ví dụ: 08:00 - 17:00)
        elif 'h' in time_str:
            time_str = time_str.replace('h', ':')
            try:
                start, end = time_str.split(' - ')
                return f'{start} - {end}'
            except ValueError:
                return np.nan
        # Xử lý các định dạng giờ khác
        else:
            return time_str.strip()
    return np.nan

df['Work Time'] = df['Work Time'].apply(standardize_time)


In [10]:
df = df.drop('Marital Status', axis=1)

In [11]:
df['Work Type'].unique()

array([nan, 'Toàn thời gian', 'Thực tập', 'Khác', 'Hợp đồng thời vụ'],
      dtype=object)

In [12]:
df

,Title,Company,Salary,Location,Job Requirements,Date Posted,Industry,Field,Experience,Education,...,Skills,Language,Nationality,Gender,Work Date,Work Type,Keyword,Industry1,Age Min,Age Max
0,Nhân Viên Trợ Lý Kế Toán (Accounting Assistant),CÔNG TY TNHH SẢN XUẤT THƯƠNG MẠI THỰC PHẨM TRƯ...,8tr-8tr ₫/tháng,"Lầu 4, 185 Bến Vân Đồn, Phường 5, Quận 4, 965-...",Khả năng giao tiếp và dịch thuật cơ bản bằng t...,2025-05-14,Kế Toán/Kiểm Toán,Hàng tiêu dùng,1,NaN,...,"Biết Tiếng Trung, Kế Toán, Tài Chính, Tiếng Anh",Bất kỳ,Không giới hạn,NaN,NaN,NaN,"Kế Toán/Kiểm Toán, Kế Toán Tài Chính, Hàng tiê...",Kế Toán Tài Chính,0,100
1,Legal Counsel (FMCG),Navigos Search,Thương lượng,"Ho Chi Minh City, Vietnam, HCMC",• Bachelor’s or Master’s Degree in Law; licens...,2025-05-14,Pháp Lý,Hàng tiêu dùng,5,Cử nhân,...,"Commercial Law, English Fluency, Regulatory Co...",Bất kỳ,Không giới hạn,Bất kỳ,Thứ 2 đến Thứ 6,Toàn thời gian,"Pháp Lý, Luật Lao động/Hưu Trí, Hàng tiêu dùng...",Luật Lao động/Hưu Trí,30,40
2,Thư Ký Tiếng Trung Bộ Phận Hành Chính,Play Travel Company Limited,"$ 500-1,000 /tháng","Tháp Bitexco, Tầng 56, 2 Đ. Hải Triều, P. Bến ...","Thành thạo tiếng Trung (nghe, nói, đọc, viết);...",2025-05-14,Hành Chính Văn Phòng,Dịch vụ lưu trú/Nhà hàng/Khách sạn/Du lịch,0,Cao đẳng,...,"Chinese, Windows, Office Software",Tiếng Trung Quốc,Không hiển thị,Nữ,NaN,Toàn thời gian,"Hành Chính Văn Phòng, Thư Ký, Dịch vụ lưu trú/...",Thư Ký,0,100
3,[Hưng Yên] 25 Kỹ Thuật Viên ( Không Yêu Cầu Ki...,Mektec Manufacturing Corporation (Vietnam) Ltd.,Thương lượng,"KCN Thăng Long II, tỉnh Hưng Yên, Plot K-3&4, ...",- Tốt nghiệp Cao đẳng trở lên\n- Không yêu cầu...,2025-05-14,Khoa Học & Kỹ Thuật,Sản xuất,0,Không giới hạn,...,Tin học cơ bản,Bất kỳ,Không hiển thị,Không hiển thị,NaN,Toàn thời gian,"Khoa Học & Kỹ Thuật, Kỹ Thuật Điện/Điện Tử, Sả...",Kỹ Thuật Điện/Điện Tử,0,100
4,Nhân Viên Kinh Doanh,Công Ty TNHH Trend Marker,$ 300-600 /tháng,"Số 9, Đường 12, Phường 10, KDC CItyland Park H...","Kỹ năng giao tiếp tốt, dễ gần và có khả năng t...",2025-05-14,Kinh Doanh,Phần Mềm CNTT/Dịch vụ Phần mềm,0,Cao đẳng,...,"Tìm Kiếm Khách Hàng, Bán Hàng, Chăm Sóc Khách ...",Bất kỳ,Không hiển thị,Không hiển thị,NaN,Toàn thời gian,"Kinh Doanh, Bán Hàng/Phát Triển Kinh Doanh, Ph...",Bán Hàng/Phát Triển Kinh Doanh,0,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,Collection Analytics Associate,Shinhan Vietnam Finance Company Limited,Thương lượng,8/F. Lottery Tower. 77 Tran Nhan Ton. D.5. HCM...,• Intermediate Graduation or higher majoring i...,2025-05-14,Ngân Hàng & Dịch Vụ Tài Chính,Tài Chính,1,Trung cấp,...,"Data Analysis, Data Administration, Collection...",Bất kỳ,Không hiển thị,Không hiển thị,NaN,Toàn thời gian,"Ngân Hàng & Dịch Vụ Tài Chính, Thu Hồi Nợ, Tài...",Thu Hồi Nợ,0,100
363,Manufacturing Logistic Operations Manager (Kor...,Samsung Electronics HCMC CE Complex,Thương lượng,"Lot I-11, Road D2, Saigon Hi-Tech Park, Tang N...",- Native Korean Speaker with University gradua...,2025-05-14,Hậu Cần/Xuất Nhập Khẩu/Kho Bãi,Điện/Điện tử,5,Cử nhân,...,"Logistic & Operation, Shipment Management, Exp...",Bất kỳ,Không hiển thị,Không hiển thị,NaN,Toàn thời gian,"Hậu Cần/Xuất Nhập Khẩu/Kho Bãi, Quản Lý Chuỗi ...",Quản Lý Chuỗi Cung Ứng,0,100
364,Digital Marketing - Google Ads - Facebook/ Met...,Văn Phòng Đại Diện Max Biocare Tại Hà Nội,"$ 700-1,000 /tháng","Level 13, ICON4 Tower 243A De La Thanh, Lang T...",Yêu cầu:\n• Kinh nghiệm: tối thiểu 12 tháng li...,2025-05-14,"Tiếp Thị, Quảng Cáo/Truyền Thông",Hàng tiêu dùng,1,Không giới hạn,...,"Google Ads, Facebook Ads, Digital Marketing, Q...",Tiếng Anh,Không hiển thị,Không hiển thị,NaN,Toàn thời gian,"Tiếp Thị, Quảng Cáo/Truyền Thông, Tiếp Thị Trự...",Tiếp Thị Trực Tuyến,24,60
365,Australian Mortgage Broker Assistant - Asset/E...,VU LE XI Data Solutions Limited Company,"$ 500-1,000 /tháng","446 Lý Thường Kiệt, phường 7, Tân Bình, Hồ Chí...",• Customer Service – speak English preferred\n...,2025-05-14,Dịc

In [13]:
data_to_show = df.iloc[:, 11]
print(data_to_show)

0                NaN
1       9:00 - 18:00
2                NaN
3                NaN
4                NaN
           ...      
362              NaN
363              NaN
364              NaN
365    08:00 - 16:00
366    08:00 - 16:00
Name: Work Time, Length: 367, dtype: object
